In [ ]:
#PEFT Finetuning the MultiModal with NeMo
## Step 1: Prerequisites:
#Clone NeMo framework with yuya2403_neva_patch. Please note that this patch is needed for NeVa PEFT Finetuning. It will be available with NeMo container in next release:

#git clone https://github.com/NVIDIA/NeMo.git

#git checkout yuya/2403_neva_patch

#Download the datase:

#Instruction Tuning Dataset The instruction tuning annotations are sourced from the LLaVA implementation and are available here.

#The associated images for the mixture instruction tuning annotations can be found here. After extracting, the data should be formatted as follows:
'''
    images
      ├── coco
      │    └── train2017
      ├── gqa
      │    └── images
      ├── ocr_vqa
      │    └── images
      ├── textvqa
      │    └── train_images
      └── vg
           ├── VG_100K
           └── VG_100K_2
'''
#The data folder will look like this:
'''
LLaVA-Instruct-mixture
├── llava_v1_5_mix665k.json
└── images
    └── ...
'''

#Download NeMo container from NGC
'''
docker run --gpus all -it --rm  -v /raid/LLaVA-Instruct-mixture:/home -v /home/NeMo:/opt/NeMo -v /home/finetune:/home --shm-size=8g -p 8888:8888 --ulimit memlock=-1 --ulimit  stack=67108864 nvcr.io/nvidia/nemo:24.03.01.framework
Here, assumption is that you have the data copied to raid/LLaVA-Instruct-mixture and this notebook is inside /home/finetune and the NeMo FW you cloned is at /home/NeMo
'''

#Download LLaVa model.
#Atleast One A100 is needed for the finetuning. Find more info on https://docs.nvidia.com/nemo-framework/user-guide/latest/multimodalmodels/multimodallanguagemodel/neva/peft.html
#This is the assumption that you are running this notebook inside the NeMo container.

In [51]:
## Step 2: Install sentencepiece
# Clone the sentencepiece repository and install it
!cd /opt && git clone https://github.com/google/sentencepiece.git

# Create the build directory and compile sentencepiece
!cd /opt/sentencepiece && mkdir build && cd build && cmake .. && make

# Install the compiled binaries and update the shared library cache
!cd /opt/sentencepiece/build && make install && ldconfig

fatal: destination path 'sentencepiece' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
[ 35%] Built target sentencepiece
[ 45%] Built target sentencepiece_train
[ 81%] Built target sentencepiece-static
[ 91%] Built target sentencepiece_train-static
[ 93%] Built target spm_encode
[ 94%] Built target spm_decode
[ 96%] Built target spm_normalize
[ 98%] Built target spm_train
[100%] Built target spm_export_vocab
Install the project...
-- Install configuration: ""
-- Up-to-date: /usr/local/lib/pkgconfig/sentencepiece.pc
-- Up-to-date: /usr/local/lib/libsentencepiece.so.0.0.0
-- Up-to-date: /usr/local/lib/libsentencepiece.so.0
-- Up-to-date: /usr/local/lib/libsentencepiece.so
-- Up-to-date: /usr/local/lib/libsentencepiece_train.so.0.0.0
-- Up-to-date: /usr/local/lib/libsentencepiece_train.so.0
-- Up-to-date: /usr/local/lib/libsentencepiece_train.so
-- Up-to-date: /usr/local/lib/libsentencepiece.a
-- Up-to-date: /usr/local/lib/libsentencepiec

In [52]:
## Step 3: Update Tokenizer
# Generate Python files from the protocol buffer definition
!cd /opt/sentencepiece/src && protoc --python_out=/opt/NeMo/scripts/tokenizers/ sentencepiece_model.proto

# Run the Python script to add special tokens for media postions in multi-modal to the sentencepiece model
!python /opt/NeMo/scripts/tokenizers/add_special_tokens_to_sentencepiece.py \
--input_file /home/llava-v1.5-13b/tokenizer.model \
--output_file /home/llava-v1.5-13b/tokenizer_neva.model \
--is_userdefined \
--tokens "<extra_id_0>" "<extra_id_1>" "<extra_id_2>" "<extra_id_3>" \
"<extra_id_4>" "<extra_id_5>" "<extra_id_6>" "<extra_id_7>"

INFO: Created token '<extra_id_0>' at ID 32000
INFO: Created token '<extra_id_1>' at ID 32001
INFO: Created token '<extra_id_2>' at ID 32002
INFO: Created token '<extra_id_3>' at ID 32003
INFO: Created token '<extra_id_4>' at ID 32004
INFO: Created token '<extra_id_5>' at ID 32005
INFO: Created token '<extra_id_6>' at ID 32006
INFO: Created token '<extra_id_7>' at ID 32007
INFO: New tokenizer vocab size: 32008
INFO: Created new tokenizer at: /home/llava-v1.5-13b/tokenizer_neva.model


In [53]:
## Step 4: Install LLaVa
# Change directory to /home
!cd /home

# Clone the LLaVa repository
!git clone https://github.com/haotian-liu/LLaVA.git

# Set the PYTHONPATH environment variable
import os
os.environ['PYTHONPATH'] = os.environ.get('PYTHONPATH', '') + ':/home/LLaVA'

fatal: destination path 'LLaVA' already exists and is not an empty directory.


In [54]:
## Step 5: convert checkpoints to .nemo
!python /opt/NeMo/examples/multimodal/multimodal_llm/neva/convert_hf_llava_to_neva.py \
 --in-file /home/llava-v1.5-13b \
 --out-file /home/llava_v1.5_13b_converted/llava-v1.5-13b-2403.nemo \
 --tokenizer-model /home/llava-v1.5-13b/tokenizer_neva.model \
 --conv-template v1

FlashAttention Installed
[NeMo W 2024-05-27 22:12:29 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo I 2024-05-27 22:12:31 convert_hf_llava_to_neva:153] loading checkpoint /home/llava-v1.5-13b
You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.
Loading checkpoint shards: 100%|██████████████████| 3/3 [00:03<00:00,  1.32s/it]
hf_config: {'vocab_size': 32000, 'max_position_embeddings': 4096, 'hidden_size': 5120, 'intermediate_size': 13824, 'num_hidden_layers': 40, 'num_attention_heads': 40, 'num_key_value_heads': 40, 'hidden_act': 'silu', 'initializer_range': 0.02, 'rms_norm_eps': 1e-05, 'pretraining_tp': 1, 'use_cache': True, 'rope_theta': 10000.0, 'rope_sca

{'precision': '${trainer.precision}', 'micro_batch_size': 16, 'global_batch_size': 128, 'tensor_model_parallel_size': 1, 'pipeline_model_parallel_size': 1, 'virtual_pipeline_model_parallel_size': None, 'restore_from_path': None, 'mm_cfg': {'llm': {'from_pretrained': None, 'freeze': False, 'model_type': 'llama_2'}, 'vision_encoder': {'from_pretrained': 'openai/clip-vit-large-patch14-336', 'from_hf': True, 'patch_dim': 14, 'hidden_size': 1024, 'vision_select_layer': -2, 'class_token_length': 1, 'freeze': True}, 'pretrain_mm_mlp_adapter': None, 'mm_mlp_adapter_type': 'mlp2x_gelu', 'use_im_start_end': False, 'model_type': 'v1'}, 'mcore_gpt': False, 'encoder_seq_length': 4096, 'max_position_embeddings': 4096, 'position_embedding_type': 'rope', 'num_layers': 40, 'hidden_size': 5120, 'ffn_hidden_size': 13824, 'num_attention_heads': 40, 'init_method_std': 0.02, 'use_scaled_init_method': True, 'hidden_dropout': 0.0, 'attention_dropout': 0.0, 'ffn_dropout': 0.0, 'kv_channels': None, 'apply_query

done layer 29
converting layer 30
done layer 30
converting layer 31
done layer 31
converting layer 32
done layer 32
converting layer 33
done layer 33
converting layer 34
done layer 34
converting layer 35
done layer 35
converting layer 36
done layer 36
converting layer 37
done layer 37
converting layer 38
done layer 38
converting layer 39
done layer 39
[NeMo W 2024-05-27 22:14:15 megatron_base_model:1146] The model: MegatronNevaModel() does not have field.name: context_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-05-27 22:14:15 megatron_base_model:1146] The model: MegatronNevaModel() does not have field.name: expert_model_parallel_size in its cfg. Add this key to cfg or config_mapping to make to make it configurable.
[NeMo W 2024-05-27 22:14:15 megatron_base_model:1146] The model: MegatronNevaModel() does not have field.name: finalize_model_grads_func in its cfg. Add this key to cfg or config_mapping to make to make it con

[NeMo I 2024-05-27 22:15:28 neva_model:554] Neva model initialized with 13017175040 trainable parameters
Unexpected key: model.language_model.embedding.word_embeddings.vision_encoder.vision_model.embeddings.class_embedding not in checkpoint but in model.
Unexpected key: model.language_model.embedding.word_embeddings.vision_encoder.vision_model.embeddings.patch_embedding.weight not in checkpoint but in model.
Unexpected key: model.language_model.embedding.word_embeddings.vision_encoder.vision_model.embeddings.position_embedding.weight not in checkpoint but in model.
Unexpected key: model.language_model.embedding.word_embeddings.vision_encoder.vision_model.pre_layrnorm.weight not in checkpoint but in model.
Unexpected key: model.language_model.embedding.word_embeddings.vision_encoder.vision_model.pre_layrnorm.bias not in checkpoint but in model.
Unexpected key: model.language_model.embedding.word_embeddings.vision_encoder.vision_model.encoder.layers.0.self_attn.k_proj.weight not in check

[NeMo I 2024-05-27 22:17:29 convert_hf_llava_to_neva:361] NeMo model saved to: /home/llava_v1.5_13b_converted/llava-v1.5-13b-2403.nemo


In [58]:
## Step 6: Run PEFT
! torchrun --nproc_per_node=1 /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_peft.py \
++cluster_type=interactive \
trainer.precision=bf16 \
model.megatron_amp_O2=True \
trainer.num_nodes=1 \
trainer.devices=1 \
trainer.val_check_interval=50 \
trainer.limit_val_batches=10 \
trainer.log_every_n_steps=1 \
trainer.max_steps=4600 \
model.micro_batch_size=2 \
model.global_batch_size=32 \
model.micro_batch_size=4 \
model.global_batch_size=32 \
model.tensor_model_parallel_size=1 \
model.pipeline_model_parallel_size=1 \
model.mcore_gpt=False \
exp_manager.create_checkpoint_callback=True \
model.data.data_path=/home/LLaVA-Instruct-mixture/llava_v1_5_mix665k_trimmed.json \
model.data.image_folder=/home/LLaVA-Instruct-mixture/images \
model.tokenizer.library=sentencepiece \
model.tokenizer.model=/home/llava-v1.5-13b/tokenizer_neva.model \
model.num_layers=40 \
model.hidden_size=5120 \
model.ffn_hidden_size=13824 \
model.num_attention_heads=40 \
model.normalization=rmsnorm \
model.data.num_workers=0 \
model.rotary_percentage=1.0 \
model.data.conv_template=llama_2 \
model.mm_cfg.vision_encoder.from_pretrained='openai/clip-vit-large-patch14' \
model.mm_cfg.llm.from_pretrained=null \
model.restore_from_path=/home/llava_v1.5_13b_converted/llava-v1.5-13b-2403.nemo \
++exp_manager.checkpoint_callback_params.save_nemo_on_train_end=True

FlashAttention Installed
[NeMo W 2024-05-27 22:38:35 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2024-05-27 22:38:38 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo I 2024-05-27 22:38:38 neva_peft:31] 
    
    ************** Experiment configuration ***********
[NeMo I 2024-05-27 22:38:38 neva_peft:32] 
    name: nemo_neva
    restore_from_path: null
    trainer:
      devices: 1
      num_nodes: 1
      accelerator: gpu
      precision: bf16
      logger: false
      enable_che

[NeMo I 2024-05-27 22:39:16 neva_model:554] Neva model initialized with 0 trainable parameters
[NeMo W 2024-05-27 22:39:47 neva_model:943] Loading state dict for MegatronNevaModel...
[NeMo W 2024-05-27 22:39:53 neva_model:947] Missing keys were detected during the load. Please double check.
[NeMo W 2024-05-27 22:39:53 neva_model:949] Missing keys: ['model.module.language_model.embedding.word_embeddings.vision_encoder.vision_model.embeddings.class_embedding', 'model.module.language_model.embedding.word_embeddings.vision_encoder.vision_model.embeddings.patch_embedding.weight', 'model.module.language_model.embedding.word_embeddings.vision_encoder.vision_model.embeddings.position_embedding.weight', 'model.module.language_model.embedding.word_embeddings.vision_encoder.vision_model.pre_layrnorm.weight', 'model.module.language_model.embedding.word_embeddings.vision_encoder.vision_model.pre_layrnorm.bias', 'model.module.language_model.embedding.word_embeddings.vision_encoder.vision_model.encod

Epoch 34: :   0%| | 0/50 [00:00<?, v_num=2, reduced_train_loss=1.070, global_ste[NeMo W 2024-05-27 22:42:32 exp_manager:196] Timer `train_step_timing` was not correctly stopped, suggesting a possible issue. The timer will be reset for now.


Epoch 40: :   0%| | 0/50 [00:00<?, v_num=2, reduced_train_loss=0.853, global_ste[NeMo W 2024-05-27 22:42:59 exp_manager:196] Timer `train_step_timing` was not correctly stopped, suggesting a possible issue. The timer will be reset for now.
^C
[2024-05-27 22:43:01,794] torch.distributed.elastic.agent.server.api: [WARNING] Received Signals.SIGINT death signal, shutting down workers
[2024-05-27 22:43:01,794] torch.distributed.elastic.multiprocessing.api: [WARNING] Sending process 27965 closing signal SIGINT
[NeMo W 2024-05-27 22:43:01 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
    


In [59]:
## Step 7 Evaluation / Inference
##Before beginning inference, ensure that you have prepared the prompt file and organized the images into a specified folder. Below is an example of how your prompt file should look:
import json

# Define the data
data = {
    "image": "000000000009.jpg",
    "prompt": "<image>\nCan you describe this image?"
}

# Define the file path
file_path = "test.jsonl"

# Write the content to the JSONL file
with open(file_path, "w") as jsonl_file:
    jsonl_file.write(json.dumps(data) + "\n")

In [63]:
import os
os.environ['PYTHONPATH'] = os.environ.get('PYTHONPATH', '') + ':/opt/NeMo'
!torchrun --nproc_per_node=1 /opt/NeMo/examples/multimodal/multimodal_llm/neva/neva_evaluation.py \
tensor_model_parallel_size=1 \
pipeline_model_parallel_size=1 \
neva_model_file=/home/nemo_experiments/nemo_neva/checkpoints/nemo_neva.nemo \
trainer.devices=1 \
trainer.precision=bf16 \
prompt_file=./test.jsonl \
inference.images_base_path=/home/LLaVA-Instruct-mixture/images/coco/train2017

FlashAttention Installed
[NeMo W 2024-05-27 22:55:40 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[NeMo W 2024-05-27 22:55:42 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
    See https://hydra.cc/docs/next/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
      ret = run_job(
    
[NeMo W 2024-05-27 22:55:43 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:563: `precision=bf16` is supported for historical reasons but its usage is discouraged. Please set your precision to bf16-mixed instead!
    
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True 

[NeMo I 2024-05-27 22:56:21 neva_model:554] Neva model initialized with 13017175040 trainable parameters
[NeMo W 2024-05-27 22:56:51 neva_model:943] Loading state dict for MegatronNevaModel...
[NeMo W 2024-05-27 22:56:57 neva_model:947] Missing keys were detected during the load. Please double check.
[NeMo W 2024-05-27 22:56:57 neva_model:949] Missing keys: ['model.language_model.embedding.word_embeddings.vision_encoder.vision_model.embeddings.class_embedding', 'model.language_model.embedding.word_embeddings.vision_encoder.vision_model.embeddings.patch_embedding.weight', 'model.language_model.embedding.word_embeddings.vision_encoder.vision_model.embeddings.position_embedding.weight', 'model.language_model.embedding.word_embeddings.vision_encoder.vision_model.pre_layrnorm.weight', 'model.language_model.embedding.word_embeddings.vision_encoder.vision_model.pre_layrnorm.bias', 'model.language_model.embedding.word_embeddings.vision_encoder.vision_model.encoder.layers.0.self_attn.k_proj.wei